<div style="background-color: #eeffee; border-color: #bbFFbb; border-left: 5px solid #88FF88; padding: 0.5em;">
Importamos las librerias necesarias.
</div>

In [1]:
import nltk
from pyspark.ml.linalg import Vector, Vectors
from pyspark.ml.clustering import LDA
from nltk.corpus import stopwords
from collections import defaultdict
import re as re
from pyspark.ml.feature import CountVectorizer , IDF
from pyspark import SparkContext
from pyspark.sql import SQLContext
import treetaggerwrapper

<div style="background-color: #eeffee; border-color: #bbFFbb; border-left: 5px solid #88FF88; padding: 0.5em;">
Cargamos los datos y nos quedamos con raw_verint, todo lo que no este vacío. Eliminamos palabras cortas y Stopwords
</div>

In [2]:
domo_dataset = sqlContext.read.parquet("dataset/domo_dataset.parquet")
raw_verint = domo_dataset.select("raw_verint").rdd.filter(lambda x: x["raw_verint"] is not None) \
              .map(lambda x: " ".join(map(lambda y: " ".join(y), x))).repartition(17)
raw_verint.cache()
StopWords = stopwords.words("spanish")
##Llamadas totales buenas
print("llamadas totales {}".format(raw_verint.count()))
numeros = ["uno", "dos", "tres", "cuatro", "cinco", "seis", "siete", "ocho", "nueve", "cero"]
tokens = raw_verint.map( lambda document: document.strip().lower()) \
	.map( lambda document: re.split(" ", document)) \
	.map( lambda word: [x for x in word if x.isalpha()]) \
	.map( lambda word: [x for x in word if len(x) > 3] ) \
	.map( lambda word: [x for x in word if x not in StopWords]) \
	.map( lambda word: [x for x in word if x not in numeros]) \
	.zipWithIndex()

llamadas totales 185109


<div style="background-color: #eeffee; border-color: #bbFFbb; border-left: 5px solid #88FF88; padding: 0.5em;">
Contamos el número de aparición de cada palabra. 
</div>

In [3]:
termCounts_tokens =  tokens.filter(lambda x: len(x)>0) \
    .flatMap(lambda document: document[0]) \
    .map(lambda word: (word, 1)) \
    .reduceByKey( lambda x,y: x + y)   \
    .map(lambda tuple: (tuple[1], tuple[0]))  \
    .sortByKey(False)

<div style="background-color: #eeffee; border-color: #bbFFbb; border-left: 5px solid #88FF88; padding: 0.5em;">
Extraemos el vocabulario. 
</div>

In [4]:
###umbral
threshold_value =50 
vocabulary_tokens = termCounts_tokens   \
  .filter(lambda x : x[0] < threshold_value)  \
  .map(lambda x: x[1])  \
  .zipWithIndex()  \
  .collectAsMap()
vocab_size_tokens = len(vocabulary_tokens)

<div style="background-color: #eeffee; border-color: #bbFFbb; border-left: 5px solid #88FF88; padding: 0.5em;">
creamos el dataframe necesario para aplicar LDA. 
</div>

In [5]:
df_txts = sqlContext.createDataFrame(tokens, ["list_of_words",'index'])
cv = CountVectorizer(inputCol="list_of_words", outputCol="features", vocabSize=vocab_size_tokens, minDF=10.0)
cvmodel = cv.fit(df_txts)
result_cv = cvmodel.transform(df_txts)
lda_df = result_cv[['index', 'features']]

<div style="background-color: #eeffee; border-color: #bbFFbb; border-left: 5px solid #88FF88; padding: 0.5em;">
Aplicamos el modelo LDA.
</div>

In [6]:
num_topics=10
max_iterations = 50
lda_tokens = LDA(k=num_topics, maxIter=max_iterations)
model_tokens =lda_tokens.fit(lda_df)


<div style="background-color: #eeffee; border-color: #bbFFbb; border-left: 5px solid #88FF88; padding: 0.5em;">
Extraemos los topics y guardamos el modelo.
</div>

In [7]:
topics = model_tokens.describeTopics()
model_tokens.save("model_lda_tokens.prueba")

<div style="background-color: #eeffee; border-color: #bbFFbb; border-left: 5px solid #88FF88; padding: 0.5em;">
Imprimimos los topics. 
</div>

In [8]:
#topics.show(truncate=False)
terms_tokens = dict([(value, key) for key, value in vocabulary_tokens.items()])
topics_term = topics.rdd.map(lambda x: [x[0],list(map(lambda y: terms_tokens[y],x[1])), x[2]])
topics_term.toDF(["t1", "t2"])["t1", "t2"].show(truncate=False)


+---+---------------------------------------------------------------------------------------------------------+
|t1 |t2                                                                                                       |
+---+---------------------------------------------------------------------------------------------------------+
|0  |[rayos, adherir, campillos, cool, aplicárselo, mensualment, presionamos, oriol, nuri, ciencuenta]        |
|1  |[campillos, cool, oriol, adherir, inversiones, bajemos, mensualment, acoso, consignado, obispo]          |
|2  |[campillos, platicar, adherir, nuri, mensualment, penalice, seque, saludado, cool, picazón]              |
|3  |[planteamos, enviaste, campillos, aplicárselo, adherir, informadas, desglosa, asaltado, saludado, antony]|
|4  |[campillos, cool, adherir, ancianos, mensualment, seque, arreglaran, saludado, penalice, aplicárselo]    |
|5  |[interpreto, saludado, campillos, desglosa, seque, cogera, solventada, aplicárselo, arreglaran, cen

<div style="background-color: #ffeeee; border-color: #ffbbbb; border-left: 5px solid #ff8888; padding: 0.5em;">
Parece que esto no tiene mucho sentido con los temas que queremos extraer de un call center. He probado con diferentes parámetros y nada...

</div>

<div style="background-color: #eeffee; border-color: #bbFFbb; border-left: 5px solid #88FF88; padding: 0.5em;">
Volvemos a aplicar el modelo sobre bigramas, en este caso nos quedamos solo con parejas Nombre-Nombre, Verbo Nombre y Nombre adjetivo. Y nos quedamos con la raiz. 
</div>

In [9]:
def is_candidate(x):
	good = [[u'VLinf', u'NC'], [u'NC', u'ADJ'], [u'NC', u'NC'] ]
	for g in good:
		if (x[0][1] == g[0] and  x[1][1] == g[1]):
			return True
	return False
	
def get_bigrams(x):
	tagger = treetaggerwrapper.TreeTagger(TAGLANG='es', TAGPARFILE="/tmp/tree/spanish.par", TAGDIR="/tmp/tree/tree-tagger-3.2.1/")
	pos_tags= map(lambda y: y.split("\t"),list(tagger.tag_text((" ".join(x[0])) )))
	bigrams = list(nltk.bigrams(pos_tags))
	candidate_bigrams = list(filter(is_candidate ,bigrams))
	stemmed_bigram = map(lambda x: x[0][2]  + " " + x[1][2],candidate_bigrams)
	tokens = [stemmed_bigram, x[1]]
	return tokens


def get_bigrams(x):
	tagger = treetaggerwrapper.TreeTagger(TAGLANG='es', TAGPARFILE="/tmp/tree/spanish.par", TAGDIR="/tmp/tree/tree-tagger-3.2.1/")
	pos_tags= list(map(lambda y: y.split("\t"),list(tagger.tag_text((" ".join(x[0])) ))))
	bigrams = list(nltk.bigrams(pos_tags))
	candidate_bigrams = list(filter(is_candidate ,bigrams))
	stemmed_bigram = list(map(lambda x: x[0][2]  + " " + x[1][2],candidate_bigrams))
	tokens = [stemmed_bigram, x[1]]
	return tokens

bigrams = tokens.map(get_bigrams)	
bigrams.cache()
bigrams.count()

185109

<div style="background-color: #eeffee; border-color: #bbFFbb; border-left: 5px solid #88FF88; padding: 0.5em;">
Contamos las ocurrencias. 
</div>

In [10]:
termCounts_bigrams =  bigrams.filter(lambda x: len(x)>0) \
    .flatMap(lambda document: document[0]) \
    .map(lambda word: (word, 1)) \
    .reduceByKey( lambda x,y: x + y)   \
    .map(lambda tuple: (tuple[1], tuple[0]))  \
    .sortByKey(False)

<div style="background-color: #eeffee; border-color: #bbFFbb; border-left: 5px solid #88FF88; padding: 0.5em;">
Extraemos el vocabulario
</div>

In [11]:
###umbral
threshold_value =15 
vocabulary_bigrams = termCounts_bigrams   \
  .filter(lambda x : x[0] < threshold_value)  \
  .map(lambda x: x[1])  \
  .zipWithIndex()  \
  .collectAsMap()
vocab_size_bigrams = len(vocabulary_bigrams)

<div style="background-color: #eeffee; border-color: #bbFFbb; border-left: 5px solid #88FF88; padding: 0.5em;">
Obtenemos el dataframe para calcular el modelo</div>

In [12]:
df_txts = sqlContext.createDataFrame(bigrams, ["list_of_words",'index'])
cv = CountVectorizer(inputCol="list_of_words", outputCol="features", vocabSize=vocab_size_bigrams, minDF=10.0)
cvmodel = cv.fit(df_txts)
result_cv = cvmodel.transform(df_txts)
lda_df = result_cv[['index', 'features']]

<div style="background-color: #eeffee; border-color: #bbFFbb; border-left: 5px solid #88FF88; padding: 0.5em;">
Aplicamos LDA</div>

In [13]:
lda_bigrams = LDA(k=num_topics, maxIter=max_iterations)
model_bigrams =lda_bigrams.fit(lda_df)
topics_bigrams = model_bigrams.describeTopics()
model_bigrams.save("model_lda_bigrams.prueba")

<div style="background-color: #eeffee; border-color: #bbFFbb; border-left: 5px solid #88FF88; padding: 0.5em;">
Mostramos los topics.
</div>

In [14]:
#topics.show(truncate=False)
terms_bigrams = dict([(value, key) for key, value in vocabulary_bigrams.items()])
topics_term = topics_bigrams.rdd.map(lambda x: [x[0],list(map(lambda y: terms_bigrams[y],x[1])), x[2]])
topics_term.toDF(["t1", "t2"])["t1", "t2"].show(truncate=False)

+---+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|t1 |t2                                                                                                                                                                     |
+---+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0  |[gusto teléfono, hacer vecino, número futbol, portabilidad marido, foco champion, tipo semanal, sánchez equipo, ruido correcto, decir moviles, tiempo devolución]      |
|1  |[mirar trámite, número futbol, ahorrar cuota, tiempo giga, emoción paso, nombre paulina, amor llamada, mes lista, tipo semanal, principio grande]                      |
|2  |[gusto teléfono, número futbol, euro clienta, abonado caso, servicio sánchez, hacer vecino, sánchez equipo, dato regalo, ruid

<div style="background-color: #ffeeee; border-color: #FFbbbb; border-left: 5px solid #FF8888; padding: 0.5em;">
Aquí parece que al menos los bigramas tienen algo más de sentido con una empresa telefónica, sin embargo no soy capaz de distinguir temas concretos y en algunos casos como el 9 parece que hablan de Barcelona (carrer, Barcelona). No sé como podemos forzar al algoritmo a descubrir los temas que nos interesan.  ¿Alguna idea?
    <BR><BR>
He probado con otros números de topics, pero los resultados no han sido mejores. 
    
</div>